In [ ]:
import os
import fitz  # PyMuPDF
import faiss
import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from baml_client import b
from baml_client.types import Document

# Load environment variables
load_dotenv()

# Load local embedding model (open source)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 1. Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# 2. Chunk text
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# 3. Get embedding (open source)
def get_embedding(text):
    return embedding_model.encode(text)

# 4. Build FAISS index
def build_faiss_index(chunks):
    sample_vector = get_embedding(chunks[0])
    dimension = len(sample_vector)
    index = faiss.IndexFlatL2(dimension)
    embeddings = np.array([get_embedding(chunk) for chunk in chunks])
    index.add(embeddings)
    return index, embeddings

# 5. Retrieve relevant chunks
def retrieve_chunks(query, chunks, index, top_k=3):
    query_embedding = get_embedding(query)
    D, I = index.search(np.array([query_embedding]), top_k)
    return [chunks[i] for i in I[0]]

# 6. Use BAML to answer the question
def answer_question_with_baml(question, relevant_chunks):
    documents = [Document(title=f"Chunk {i+1}", content=chunk) for i, chunk in enumerate(relevant_chunks)]
    response = b.AnswerQuestion(question=question, documents=documents)
    return response

# === Main Flow ===
if __name__ == "__main__":
    pdf_path = "your_file.pdf"  # Replace with your file
    text = extract_text_from_pdf(pdf_path)
    chunks = chunk_text(text)
    index, _ = build_faiss_index(chunks)

    query = "What is the main idea of the document?"
    relevant_chunks = retrieve_chunks(query, chunks, index)
    answer = answer_question_with_baml(query, relevant_chunks)

    print("\n🧠 Answer:", answer)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}